**A deep autoencoder based recommendation system for CareerVillage.org**

*  Overview

The following kernel is an entry to the "Data Science for Good: CareerVillage.org" kaggle competition and describes a method to match questions to professionals motivated to answer them.
Divided into five steps, tokenization, autoencoding, clustering, prediction and recommendation.

* Method description

The proposed method is built upon the following general assumptions: questions and answers share similar terms, engagement by volunteers is proportional to the number of answers posted by them and volunteer information is enough to determinate their expertise. The following summarizes the main steps.

* **Tokenization** 
From Standford NLP page "Given a character sequence and a defined document unit, tokenization is the task of chopping it up into pieces, called tokens, often loosely referred to as terms or words". To perform the tokenization unique words from questions_body data set are selected. Then only the nouns from that initial token are taken and each word stem is taken as the token first preprocessed token. And finally, the frequency of each token over each question is measured and only those words with a frequency over 15 (15 is referred in the code as CropFactor) times the global minimum frequency are taken as the final token. 

* **Autoencoding **
Using the final token, the frequencies over each token and each question is measured, leaving a (samples, tokens) size matrix. As the resulting data set is highly dimensional, to reduce the amount of data needed to perform the recommendation a dimensionality reduction technique known as autoencoder is used. From Wikipedia "An autoencoder is a type of artificial neural network used to learn efficient data codings in an unsupervised manner The aim of an autoencoder is to learn a representation (encoding) for a set of data, typically for dimensionality reduction". 
The resulting high dimensional matrix is then auto-encoded with twelve layers under complete deep autoencoder, with a bottleneck size of 3, leaving a (samples,3) matrix. 

* **Clustering**
With a low dimensional data set searching for similar questions is an easier task, also known as clustering. Defined as "the task of grouping a set of objects in such a way that objects in the same group (called a cluster) are more similar (in some sense) to each other than to those in other groups  (clusters). From the dimensionally reduced matrix, the number of clusters is calculated by a combination of the elbow method and the silhouette factor. That will ensure the minimum number of clusters with the highest accuracy.

* **Prediction**
By determining the clusters in the dimensionally reduced data set, a number of labels are obtained and those labels can be used for classification. With the labels obtained from the clustering step and the encoded matrix, a random forest classifier is trained, to predict the cluster of each data point and to predict future datasets without the need of the clustering step.

* **Recommendation**
Finally, all the answers have a similar process. Token frequencies are calculated, encoded using the already pre-trained model and classified with the pre-trained random forest classifier. Then the frequency of each unique author answers a question of each predicted cluster is measured. The first recommendation function suggests five authors with the highest frequency to answer a question from a given cluster. 

* **Recommendation Extension**
The first batch of recommendations will represent the most likely volunteers that will answer a question. To extend the recommendations to those volunteers without answers, data provided by the volunteers from the professionals dataset is processed as described in the recommendation step. And five additional volunteers are suggested that based on the information provided by themselves. The final recommendation function will check for volunteer duplicity and in those cases will change it to a different volunteer.


*  Method implementation 

The intended use for the proposed method is as follows: once a question is posted over CareerVillage.org, such question is classified and the resulting label is then passed to the final recommendation function that returns ten possible volunteers to answer that question. All the trained models are generated by the use of well documented and popular machine learning libraries, additional data can be added to the models regularly.

*  Method Extensibility

In terms of extensibility, the autoencoding of the questions can also be used for population segmentation for both students and volunteers. By segmenting the population new tags can be proposed, or a tag suggestion system can be implemented based on the respective population. Also, activity, time to response or some other metrics can be measured for each population to get a better awareness of the community.


* Code

For simplicity over each code cell the markdown cell will contain a general description of the process performed by the code, and over the code cell a detailed description over each element if is needed.

In [ ]:
#importing the libraries to use 
import os
import re
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn import preprocessing as pr
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


from keras import initializers
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.callbacks import ReduceLROnPlateau

globalSeed=50
from numpy.random import seed
seed(globalSeed)
from tensorflow import set_random_seed
set_random_seed(globalSeed)

In [ ]:
#General plot style
def PlotStyle(Axes,Title,x_label,y_label):
    
    Axes.spines['top'].set_visible(False)
    Axes.spines['right'].set_visible(False)
    Axes.spines['bottom'].set_visible(True)
    Axes.spines['left'].set_visible(True)
    Axes.xaxis.set_tick_params(labelsize=12)
    Axes.yaxis.set_tick_params(labelsize=12)
    Axes.set_ylabel(y_label,fontsize=14)
    Axes.set_xlabel(x_label,fontsize=14)
    Axes.set_title(Title)
    

First loading the data

In [ ]:
AnswersData=pd.read_csv(r"../input/answers.csv")
QuestionsData=pd.read_csv(r"../input/questions.csv")

AnswersData=AnswersData.drop(AnswersData.index[48625]) #While exploring the data that answer body is equal to 'Nan'

Working with only the text.

In [ ]:
AnswersBody=np.array(AnswersData['answers_body'])
QuestionsBody=np.array(QuestionsData['questions_body'])
QuestionsTitle=np.array(QuestionsData['questions_title'])

Some functions to remove some artifacts of the web scrapping 

In [ ]:
def TextHTMLRemoval(TargetString):
  #Removes HTML tags from the text'
  characterList=['<p>','</p>','<br>','<ol>','<li>','</li>','h1&gt','&lt;','\n','\r','href','html','http','https']
  cTarg=TargetString
  
  for val in characterList:
    cTarg=re.sub(val,' ',cTarg)
    
  return cTarg
  
def GetTextOnly(TargetString):
  #Removes numbers and punctuation from the text
  cTarg=TargetString
  cTarg=re.sub(r'[^\w\s]',' ',cTarg)
  cTarg=re.sub(r'[0-9]+', ' ', cTarg)
  
  return cTarg
  
def MakeTextTransform(TargetString):
  #Wrapper function 
  cTarg=TargetString
  cTarg=TextHTMLRemoval(cTarg)
  cTarg=GetTextOnly(cTarg)
  
  return cTarg.lower()

Cleaning the text with the wrapper function

In [ ]:
FiltAnswers=[MakeTextTransform(val) for val in AnswersBody]
FiltQuestionsB=[MakeTextTransform(val) for val in QuestionsBody]

del AnswersBody,QuestionsBody,QuestionsTitle

Selecting only the unique words in the text

In [ ]:
def UniqueDataSetTokens(TextData):
  
  cData=TextData
  nData=len(cData)
    
  def SplitAndReduce(TargetString):
    return list(set(TargetString.split()))#returns the unique text elements
  
  container=SplitAndReduce(cData[0])
  
  for k in range(1,nData):
    container=container+SplitAndReduce(cData[k])
    if k%100==0:  #only each 100 steps the container is transformed to a set to eliminate duplicates
      container=list(set(container))
  
  return container


Selecting only the nouns from the text

In [ ]:
def TokenProcessing(TokenData):
  
  cToken=TokenData
  s=stopwords.words('english')
  cToken=list(set(cToken)-set(s)) #Eliminates stop words
   
  PosTagging=nltk.pos_tag(cToken) #Selecting only nouns using part of speech tagging
  lToken=[]
  for val in  PosTagging:
    if val[1]=='NN':
      lToken.append(val[0])
  
  localps=PorterStemmer()
  localToken=[]
  for val in lToken:
    localToken.append(localps.stem(val))#stem of each token 
  
  stemS=[]
  for val in s:
    stemS.append(localps.stem(val))
  
  localToken=list(set(localToken))
  localTok=[val for val in localToken if 16>len(val)>4]#eliminating words of small size and those greater than 16 characters
  
  return localTok

Counting the frequencies of the token terms 

In [ ]:
def TokenFrequencies(TargetString,Token):
  
  cST0=TargetString.split()
  localPS=PorterStemmer()
  cST=[]
  for val in cST0:
    cST.append(localPS.stem(val))#changes each word in the text to the stem word
  
  cTok=Token
  nToken=len(cTok)
  Container=[0 for k in range(nToken)]#frequencies of each token 
  cDict={}
  
  for k in range(nToken):
    cDict[cTok[k]]=k #returns the index over the container 
    
  for val in cST:
    try: 
      cval=cDict[val]
      Container[cval]=Container[cval]+1 
    except KeyError: 
      pass
  
  return np.array(Container)*(1/(sum(Container)+1))#adding one to avoid division by zero error 


Calculating the distribution of the tokens

In [ ]:
TargetDataSet=FiltQuestionsB #Data set used to generate the autoencoder can be changed to QuestionsTitle or AnswersBody 
 
TargetTokens=UniqueDataSetTokens(TargetDataSet)   
stemToken=TokenProcessing(TargetTokens)

TargetFreq=[TokenFrequencies(val,stemToken) for val in TargetDataSet]
TargetFreq=np.reshape(TargetFreq,(len(TargetFreq),len(TargetFreq[0])))
sumFreqs=TargetFreq.sum(axis=0)

plt.figure(1)
plt.hist(sumFreqs,bins=200)
plt.yscale('log',nonposy='clip')
ax=plt.gca()
PlotStyle(ax,'','Token Frequency','Frequency Counts')

Cropping the least frequent tokens 

In [ ]:
CropFactor=15
SelectedIndex=[j for j in range(sumFreqs.size) if sumFreqs[j]>CropFactor*sumFreqs.min()]
FinalToken=[stemToken[val] for val in SelectedIndex]

Frequencies of the final token over the target data set and scaling of the data matrix

In [ ]:
TargetFreq=[TokenFrequencies(val,FinalToken) for val in TargetDataSet]
TargetSc=pr.MinMaxScaler()#MinMaxScaling of the target data
TargetSc.fit(TargetFreq)  
normData=TargetSc.transform(TargetFreq)

Deep autoencoding of the token frequencies 

In [ ]:
fixedSeed=2017

inputShape=normData[0].size

ae = Sequential()
ae.add(Dense(int(inputShape/3),  activation='elu',
            kernel_initializer=initializers.glorot_uniform(seed=fixedSeed),
            bias_initializer=initializers.glorot_uniform(seed=fixedSeed),
            input_shape=(inputShape,)))

ae.add(Dense(380,  activation='sigmoid',
            kernel_initializer=initializers.glorot_uniform(seed=fixedSeed),
            bias_initializer=initializers.glorot_uniform(seed=fixedSeed)))

ae.add(Dense(190,  activation='sigmoid',
            kernel_initializer=initializers.glorot_uniform(seed=fixedSeed),
            bias_initializer=initializers.glorot_uniform(seed=fixedSeed)))

ae.add(Dense(45,  activation='sigmoid',
            kernel_initializer=initializers.glorot_uniform(seed=fixedSeed),
            bias_initializer=initializers.glorot_uniform(seed=fixedSeed)))

ae.add(Dense(12,  activation='elu',
            kernel_initializer=initializers.glorot_uniform(seed=fixedSeed),
            bias_initializer=initializers.glorot_uniform(seed=fixedSeed)))

ae.add(Dense(3,    activation='linear', name="bottleneck",
            kernel_initializer=initializers.glorot_uniform(seed=fixedSeed),
            bias_initializer=initializers.glorot_uniform(seed=fixedSeed)))

ae.add(Dense(12,  activation='elu',
            kernel_initializer=initializers.glorot_uniform(seed=fixedSeed),
            bias_initializer=initializers.glorot_uniform(seed=fixedSeed)))

ae.add(Dense(45,  activation='sigmoid',
            kernel_initializer=initializers.glorot_uniform(seed=fixedSeed),
            bias_initializer=initializers.glorot_uniform(seed=fixedSeed)))

ae.add(Dense(190,  activation='sigmoid',
            kernel_initializer=initializers.glorot_uniform(seed=fixedSeed),
            bias_initializer=initializers.glorot_uniform(seed=fixedSeed)))

ae.add(Dense(380,  activation='sigmoid',
            kernel_initializer=initializers.glorot_uniform(seed=fixedSeed),
            bias_initializer=initializers.glorot_uniform(seed=fixedSeed)))

ae.add(Dense(int(inputShape/3),  activation='elu',
            kernel_initializer=initializers.glorot_uniform(seed=fixedSeed),
            bias_initializer=initializers.glorot_uniform(seed=fixedSeed)))

ae.add(Dense(inputShape,  activation='linear',
            kernel_initializer=initializers.glorot_uniform(seed=fixedSeed),
            bias_initializer=initializers.glorot_uniform(seed=fixedSeed)))

decay=0.001
reduce_lr=ReduceLROnPlateau(monitor='loss',factor=0.1,patience=5,min_lr=decay)    

ae.compile(loss='mean_squared_error', optimizer = Adam(lr=0.01,decay=decay))
ae.fit(normData, normData, batch_size=3024, epochs=50, verbose=1,callbacks=[reduce_lr])

encoder=Model(ae.input, ae.get_layer('bottleneck').output)
Zenc=encoder.predict(normData)


Visualization of the bottleneck representation 

In [ ]:
EncScaler=pr.MinMaxScaler() #Min Max scaling of the encoded data
EncScaler.fit(Zenc)
EncoderScaled=EncScaler.transform(Zenc)

fig = plt.figure(2,figsize=(8,8))

ax0=fig.add_subplot(2, 2, 1)
ax0.plot(EncoderScaled[:,0],EncoderScaled[:,1],'bo',alpha=0.0125)
PlotStyle(ax0,'','Encoded Dimension 1','Encoded Dimension 2')

ax1=fig.add_subplot(2, 2, 2)
ax1.plot(EncoderScaled[:,0],EncoderScaled[:,2],'bo',alpha=0.0125)
PlotStyle(ax1,'','Encoded Dimension 1','Encoded Dimension 3')

ax2=fig.add_subplot(2, 2, 3)
ax2.plot(EncoderScaled[:,1],EncoderScaled[:,2],'bo',alpha=0.0125)
PlotStyle(ax2,'','Encoded Dimension 2','Encoded Dimension 3')

ax3=fig.add_subplot(2, 2, 4,projection='3d')
ax3.scatter(EncoderScaled[:,0],EncoderScaled[:,1],EncoderScaled[:,2],color='b',alpha=0.05)

plt.tight_layout()

Determination of the optimal number of clusters 

In [ ]:
kmeansInertia=[]
fitness=[]
xv=[]

for k in range(5,60,3):
  
  cluster=KMeans(n_clusters=k)
  cluster.fit(EncoderScaled)
  labels =cluster.labels_
  kmeansInertia.append(cluster.inertia_)
  fitness.append(metrics.silhouette_score(EncoderScaled, labels) )
  xv.append(k)

plt.figure(3)
plt.plot(xv,kmeansInertia/max(kmeansInertia),label='SSE/max(SSE)')
plt.plot(xv,fitness/max(fitness),label='Silhouette Score')
plt.legend(loc=0)
ax=plt.gca()
PlotStyle(ax,'','k-clusters','Normalized performance')

optnClusters=xv[np.argmax(fitness[3:len(fitness)])]
OptimalKMeans=KMeans(n_clusters=optnClusters)
OptimalKMeans.fit(EncoderScaled)
YVals=OptimalKMeans.labels_

Visualization of some of the question in the generated clusters 

In [ ]:
RandomQuestTSample=np.random.randint(0,len(YVals),5)

for val in RandomQuestTSample:
  
  print('Cluster number = ' + str(YVals[val]))
  print(FiltQuestionsB[val])

Training of a random forest classifier for further predictions 

In [ ]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(EncoderScaled, YVals, train_size = 0.75,test_size=0.25, random_state = 23)

RFC=RandomForestClassifier(n_estimators=100)
RFC.fit(Xtrain,Ytrain)

yPred=RFC.predict(Xtest)
YTest=np.reshape(np.array(Ytest),(yPred.shape))

cm=metrics.confusion_matrix(YTest,yPred)
ncm=cm/cm.sum(axis=1)

del Xtrain, Xtest, Ytrain, Ytest, yPred

plt.figure(4)
ax=plt.gca()
im=ax.imshow(ncm,interpolation='nearest',cmap=plt.cm.Blues)
ax.figure.colorbar(im,ax=ax)

Processing of the answers data 

In [ ]:
AnswersTokFreq=[TokenFrequencies(val,FinalToken) for val in FiltAnswers]
normAnswerData=TargetSc.transform(AnswersTokFreq)
EncAnswers=encoder.predict(normAnswerData)
EncScaledAnswers=EncScaler.transform(EncAnswers)
AnswersClasses=RFC.predict(EncScaledAnswers)

Visualization of some of the clusters

In [ ]:
RandomClusters=np.random.randint(0,optnClusters,2)

for val in RandomClusters:
  
  print('---------------------------------------------')
  print('Cluster Number = ' + str(val))
  for j in range(len(YVals)):
    if YVals[j]==val:
      print('----------------Question----------------')
      print(FiltQuestionsB[j])
      break
    
  for j in range(len(AnswersClasses)):
    if AnswersClasses[j]==val:
      print('----------------Answer----------------')
      print(FiltAnswers[j])
      break

Author characteristics

In [ ]:
def GetAuthorFrequency(Authors,UniqueAuthors):
  
  cAuth=Authors
  uAuth=UniqueAuthors
  nAuthors=len(uAuth)
  
  localFreq=[0 for k in range(nAuthors)]#frequencies of each author
  localDict={}

  for j in range(nAuthors):
    localDict[uAuth[j]]=j#returns the index over localFreq
  
  for val in cAuth:
    localFreq[localDict[val]]=localFreq[localDict[val]]+1
  
  return np.array(localFreq)/len(cAuth)

def AuthorClusterCorrespondance(AnswerID,UniqueAuthors,AnswerClasses):
  
  cAnswID=AnswerID
  uAuth=UniqueAuthors  
  cClass=AnswerClasses
  nAuthors=len(uAuth)
  nData=len(cClass)
  
  localMatrix=np.zeros((nAuthors,optnClusters))#frequencies of each author over each cluster
  localDict={}

  for j in range(nAuthors):
  
    localDict[uAuth[j]]=j#returns the index over localMatrix
    
  for j in range(nData):
    
    AuthLoc=localDict[cAnswID[j]]
    ClassLoc=cClass[j]
    localMatrix[AuthLoc,ClassLoc]=localMatrix[AuthLoc,ClassLoc]+1
    
  return localMatrix/len(YVals)

AnswersAuthorID=np.array(AnswersData['answers_author_id'])
UniqueAuthors=np.unique(AnswersAuthorID)

AuthorFreq=GetAuthorFrequency(AnswersAuthorID,UniqueAuthors)
AuthorMatrix=AuthorClusterCorrespondance(AnswersAuthorID,UniqueAuthors,AnswersClasses)

plt.figure(6)
fig,axes=plt.subplots(1,2,figsize=(10,4))

axes[0].plot(AuthorFreq)
PlotStyle(axes[0],'','Unique Authors','Answer Probability')

axes[1].hist(AuthorFreq,bins=40)
axes[1].set_yscale('log')
PlotStyle(axes[1],'','Answer Probability','Probability Counts')


Basic recommendation function 

In [ ]:
def GetMostLikelyAuthors(ClusterNumber,UniqueAuthors,AuthorMatrix):
  #Wrapper function for the first recomendation
  cData=AuthorMatrix[:,ClusterNumber]
  ind=np.argsort(cData) #sorting of the data
  lAuthors=[UniqueAuthors[ind[-1]],UniqueAuthors[ind[-2]],UniqueAuthors[ind[-3]],UniqueAuthors[ind[-4]],UniqueAuthors[ind[-5]]]
  
  return lAuthors

def FindLocations(value,List):
  
  cval=value
  clis=List
  cont=[]
    
  for k in range(len(clis)):
    if cval==clis[k]:
      cont.append(k)
      
  return cont

Probability to get an answer from the recommended authors 

In [ ]:
QuestionID=np.array(QuestionsData['questions_id'])
AnswerQuestionID=np.array(AnswersData['answers_question_id'])
AnswProb=[]

for k in range(len(YVals)):
  
  cClust=YVals[k]
  
  cRec=GetMostLikelyAuthors(cClust,UniqueAuthors,AuthorMatrix)
  locs=FindLocations(QuestionID[k],AnswerQuestionID)
  kL=[0,0,0,0,0]
  
  for val in locs:
    
   cAid=AnswersAuthorID[val]
   
   if cAid==cRec[0]:
     kL=[1,0,0,0,0]
   elif cAid==cRec[1]:
      kL=[0,1,0,0,0]
   elif cAid==cRec[2]:
      kL=[0,0,1,0,0]
   elif cAid==cRec[3]:
      kL=[0,0,0,1,0]
   elif cAid==cRec[4]:
      kL=[0,0,0,0,1]
       
  AnswProb.append(kL)

AnswProb=np.array(AnswProb)

fig,axes=plt.subplots(1,2,figsize=(10,4))

axes[0].plot(AnswProb.sum(axis=0)/len(YVals))
PlotStyle(axes[0],'','Recomendations','Answer Probability')

axes[1].plot(np.cumsum(AnswProb.sum(axis=0))/len(YVals))
PlotStyle(axes[1],'','Recomendations','Cumulative Answer Probability')

The probability to get an answer for the first recommendation is around 0.06, almost twice the probability of the most prolific author, the combined probability of all the recommendations is around 0.2, almost twice the probability of all the most prolific authors combined. 

Merging the data over the professionals dataset

In [ ]:
ProfesionalsData=pd.read_csv(r"../input/professionals.csv")
VolunteersData=ProfesionalsData.dropna()
VolunteersData['variance'] = VolunteersData['professionals_industry'] +' ' +VolunteersData['professionals_headline']

VolunteersID=np.array(VolunteersData['professionals_id'])
VolunteersText=np.array(VolunteersData['variance'])

Filtering the merged text

In [ ]:
FiltVolunteers=[MakeTextTransform(val) for val in VolunteersText]

Classification of the volunteers

In [ ]:
VolunteersTokFreq=[TokenFrequencies(val,FinalToken) for val in FiltVolunteers]
normVolunteersData=TargetSc.transform(VolunteersTokFreq)
EncVolunteers=encoder.predict(normVolunteersData)
EncScaledVolunteers=EncScaler.transform(EncVolunteers)
VolunteersClasses=RFC.predict(EncScaledVolunteers)

Recommendation of volunteers

In [ ]:
def GetLikelyVolunteers(cluster,VolunteersID,VolunteersClasses):
  
  VolID=VolunteersID
  VolClas=VolunteersClasses
  cClust=cluster
  
  nData=len(VolID)
  cont=[]
  for j in range(nData):  

    cVol=VolClas[j]
    if cVol==cClust:
      
      cont.append(j)
  
  if len(cont)<=20:  
    inxs=cont
  else:  
    inxs=np.random.choice(cont,20)
    
  vol=VolunteersID[tuple([inxs])]
  
  return list(vol)

Wrapper for final recommendations

In [ ]:
def FinalRecomendation(cluster,UniqueAuthors,AuthorMatrix,VolunteersID,VolunteersClasses):
  #Wrapper function for the final recomendation 
  cClus=cluster
  UA=UniqueAuthors
  AM=AuthorMatrix
  VolID=VolunteersID
  VolC=VolunteersClasses
  
  Rec0=GetMostLikelyAuthors(cClus,UA,AM)
  Rec1=GetLikelyVolunteers(cClus,VolID,VolC)
  
  NonOverlapping=list(set(Rec1)-set(Rec0))
  
  for k in range(5):
    Rec0.append(NonOverlapping[k])
    
  return Rec0

Printing the then final recomendations 

In [ ]:
print(FinalRecomendation(3,UniqueAuthors,AuthorMatrix,VolunteersID,VolunteersClasses))